In [1]:
# import library and framework
import pandas as pd
from sklearn.svm import SVC
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import cv2
import mediapipe as mp
import os
import pickle
from moviepy.editor import VideoFileClip

In [9]:
with open('model/body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [10]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [12]:
cap = cv2.VideoCapture("video/rakaat.mp4")
clip = VideoFileClip("video/rakaat.mp4")
hasil = []

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
#         try:
            # Extract Pose landmarks
#         pose = results.pose_landmarks.landmark
#         pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
#         face = results.face_landmarks.landmark
#         face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
        # Concate rows
        pose = results.pose_landmarks.landmark
        centering = []
        pose_row = []
        n=0

        for landmark in pose:
            x = list(np.array([landmark.x, landmark.y, landmark.z, landmark.visibility]).flatten())
            for y in x:
                pose_row.append(y)
                if y == x[0]:
                    a = list(np.array([pose[0].x - y]))
                    for b in a:
                        centering.append(b)
                if y == x[1]:
                    a = list(np.array([pose[0].y - y]))
                    for b in a:
                        centering.append(b)
                if y == x[2]:
                    a = list(np.array([pose[0].z - y]).flatten())
                    for b in a:
                        centering.append(b)
        row = centering#+face_row
        

    #             # Append class name 
    #             row.insert(0, class_name)

    #             # Export to CSV
    #             with open('coords.csv', mode='a', newline='') as f:
    #                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    #                 csv_writer.writerow(row) 

                # Make Detections
        X = pd.DataFrame([row])
        body_language_class = model.predict(X)[0]
        body_language_prob = model.predict_proba(X)[0]
#         print(body_language_class, body_language_prob)
        
        urutan = ['Berdiri', 'Rukuk', 'Itidal', 'Sujud', 'Takhiyat Akhir', 'Sujud']
        rakaat = 0
        gerakan = 6
        jml = 5
        n = 0
        
        hasil.append(body_language_class)
        for i in range(len(hasil)):
            if i <= jml:
                if hasil[i] == urutan[n]:
                    if i <= jml:
                        gerakan -= 1
                        jml += 5
                        n+=1
                        if gerakan == 0:
                            rakaat += 1
                            gerakan += 6
                            n -= 6
                else:
                    jml += 1
                
        print(gerakan)
        
#         while len(hasil) >= 10:
            
                    
                # Grab ear coords
#         coords = tuple(np.multiply(
#                         np.array(
#                             (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
#                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
#                     , [640,480]).astype(int))

#         cv2.rectangle(image, 
#                         (coords[0], coords[1]+5), 
#                         (coords[0]+len(body_language_class)*20, coords[1]-30), 
#                         (245, 117, 16), -1)
#         cv2.putText(image, body_language_class, coords, 
#                     cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Get status box
        cv2.rectangle(image, (0,0), (220, 180), (245, 117, 16), -1)

                # Display Class
        cv2.putText(image, 'CLASS'
                    , (15,132), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, body_language_class
                    , (10,163), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display Probability
        cv2.putText(image, 'PROBABILITY'
                    , (15,24), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                    , (10,53), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Display Probability
        cv2.putText(image, 'RAKAAT'
                    , (15,76), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(rakaat)
                    , (10,107), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

#         except:
#             pass           
        cv2.imshow('Raw Webcam Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            duration = (clip.duration)
            minutes = int(duration)/60
            seconds = duration-minutes
            print(int(minutes),",",seconds)
            break

cap.release()
cv2.destroyAllWindows()

6
6
6
6
6
6
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
